In [16]:
import pandas as pd
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import dcc
from dash import html

import ibmdata

In [40]:
def generate_query(DAYSBACK, FC):
    
    if FC == 'all':
        FC = "'X2','Q6','Q5','XQ'"
    else:
        FC = f"'{FC}'"
    return f"""
    WITH lot_name_cte AS(
        SELECT DISTINCT wafer_id, E.CELL_NAME AS Lot_Name
        FROM DMIW_SYSTEMS.WAFER A,
            DMIW_SYSTEMS.PRODUCT B,
            DMIW_SYSTEMS.EWR_EVENT_FACT C,
            DMIW_SYSTEMS.EWR_EVENT D,
            DMIW_SYSTEMS.EWR_CELL E
        WHERE A.WAFERKEY=C.WAFERKEY
            AND B.PRODUCTKEY=C.PRODUCTKEY
            AND  C.EWR_CELLKEY=E.EWR_CELLKEY
            AND OWNER_USER_ID='hai.zhu@ibm.com'
            AND EWR_NAME IN ('7HPP_Name_P10', '7HPP_Name_zA')
            AND (CELL_ID LIKE '7HPP_Name_P10%' OR CELL_ID LIKE '7HPP_Name_zA%')
            AND E.CELL_NAME NOT IN ('Lot0-1','Lot0-2','uncertain')
            AND B.FAMILY_CODE IN ({FC})
    ),
    sail_cte AS(
        SELECT LEFT(wafer_id,5) AS lot_id_base, lot_id, wafer_id, family_code, 
          MIN(tw.Last_test_date) AS Sail_date, 1 AS Sail_n,
          AVG(CASE WHEN parm_Label = 'SAIL23_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p50to1p05,
          AVG(CASE WHEN parm_Label = 'SAIL23_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p65to1p05,
          AVG(CASE WHEN parm_Label = 'SAIL2_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL2_0p50to1p05,
          AVG(CASE WHEN parm_Label = 'SAIL2_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL2_0p65to1p05,
          AVG(CASE WHEN parm_Label = 'SAIL3_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL3_0p50to1p05,
          AVG(CASE WHEN parm_Label = 'SAIL3_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL3_0p65to1p05
        FROM DMIW.PTileWaferFact ptwf
        INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testparmkey = tp.testparmkey
        INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
        WHERE tw.Last_test_date >= (current date - {DAYSBACK} days)
          AND tw.Tech_id = '7HPP'
          AND tw.Level = 'H2'
          AND tw.Calcdefs in ('SSL11', 'SSL21', 'SSL31')
          AND weighted_Mean IS NOT null AND ABS(weighted_Mean) < 1e25
          AND UCASE(tp.parm_Label) LIKE '%_PERFECTYIELD'
          AND FAMILY_CODE IN ({FC})
        GROUP BY lot_id, wafer_id, family_code
        ORDER BY Sail_date desc, lot_id, wafer_id
    ),
    pele_cte AS(
        SELECT lot_Id, wafer_Id, family_code, MIN(last_Test_Date) AS Pele_date, 1 as Pele_n,
           AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_L3_HL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_L3_Vmax_PY,
           AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_L3_NL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_L3_Vnom_PY,
           AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_L3_LL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_L3_Vmin_PY,
           AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_1K_HL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_1K_Vmax_PY,
           AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_1K_NL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_1K_Vnom_PY,
           AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_1K_LL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_1K_Vmin_PY
        FROM DMIW.PTileWaferFact ptwf
        INNER JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = ptwf.testParmKey 
        INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
        WHERE Last_test_date >= (current date - {DAYSBACK} days) AND
            Tech_id = '7HPP' and Level = 'H2' AND tw.Calcdefs IN ('PEL11', 'PEL21', 'PEL31') AND
            weighted_Mean IS NOT null AND abs(weighted_Mean) < 1e25 AND
            (ucase(tp.parm_Label) LIKE 'PELE%_H2_%PERFECTYIELD')
            AND FAMILY_CODE IN ({FC})
        GROUP BY lot_Id, wafer_Id, family_code
    )
    SELECT ln.lot_name, s.lot_id_base, s.lot_id, s.family_code, Sail_date, Sail_n, s.wafer_id, 
        SAILALL_0p50to1p05, SAILALL_0p65to1p05,
        SAIL2_0p50to1p05, SAIL2_0p65to1p05, SAIL3_0p50to1p05, SAIL3_0p65to1p05,
        Pele_date, Pele_n, s.wafer_id AS wafer_id_2,
        PELE1_L3_Vmax_PY, PELE1_L3_Vnom_PY, PELE1_L3_Vmin_PY,
        PELE1_1K_Vmax_PY, PELE1_1K_Vnom_PY, PELE1_1K_Vmin_PY
    FROM sail_cte s
    FULL OUTER JOIN pele_cte p ON s.wafer_id = p.wafer_id
    LEFT JOIN lot_name_cte ln ON s.wafer_id = ln.wafer_id
    ORDER BY Sail_date desc, s.lot_id, s.wafer_id
    """

In [41]:
def get_wafer_data(DAYSBACK, FC):
    w_df = ibmdata.isdw.query(generate_query(DAYSBACK, FC))

    w_df = w_df.rename(columns={"wafer_id_2": "wafer_id"})
    w_df["sailall_0p50to1p05"] = w_df["sailall_0p50to1p05"].astype(float).round(2)
    w_df["sailall_0p65to1p05"] = w_df["sailall_0p65to1p05"].astype(float).round(2)
    w_df["sail2_0p50to1p05"] = w_df["sail2_0p50to1p05"].astype(float).round(2)
    w_df["sail2_0p65to1p05"] = w_df["sail2_0p65to1p05"].astype(float).round(2)
    w_df["sail3_0p50to1p05"] = w_df["sail3_0p50to1p05"].astype(float).round(2)
    w_df["sail3_0p65to1p05"] = w_df["sail3_0p65to1p05"].astype(float).round(2)

    w_df["pele1_l3_vmax_py"] = w_df["pele1_l3_vmax_py"].astype(float).round(2)
    w_df["pele1_l3_vnom_py"] = w_df["pele1_l3_vnom_py"].astype(float).round(2)
    w_df["pele1_l3_vmin_py"] = w_df["pele1_l3_vmin_py"].astype(float).round(2)
    w_df["pele1_1k_vmax_py"] = w_df["pele1_1k_vmax_py"].astype(float).round(2)
    w_df["pele1_1k_vnom_py"] = w_df["pele1_1k_vnom_py"].astype(float).round(2)
    w_df["pele1_1k_vmin_py"] = w_df["pele1_1k_vmin_py"].astype(float).round(2)

    return w_df.sort_values("sail_date", ascending=False)

In [44]:
get_wafer_data(20, 'all')

2023-02-28 13:05:13,215 INFO     ibmdata   : retrieved (365, 22) (rows, cols) of data in 0:00:03.550386 from ISDW


,lot_name,lot_id_base,lot_id,family_code,sail_date,sail_n,wafer_id,sailall_0p50to1p05,sailall_0p65to1p05,sail2_0p50to1p05,sail2_0p65to1p05,sail3_0p50to1p05,sail3_0p65to1p05,pele_date,pele_n,wafer_id,pele1_l3_vmax_py,pele1_l3_vnom_py,pele1_l3_vmin_py,pele1_1k_vmax_py,pele1_1k_vnom_py,pele1_1k_vmin_py
15,None,A32TK,A32TK.1,Q6,2023-02-26,1.0,A32TK-11,96.25,100.00,97.75,100.00,98.9,100.0,2023-02-26,1,A32TK-11,100.00,100.00,100.00,100.00,98.87,97.75
22,None,A32TK,A32TK.1,Q6,2023-02-26,1.0,A32TK-20,98.75,100.00,98.87,100.00,100.0,100.0,2023-02-26,1,A32TK-20,100.00,100.00,98.87,100.00,98.87,97.75
16,None,A32TK,A32TK.1,Q6,2023-02-26,1.0,A32TK-12,97.50,98.75,100.00,100.00,97.8,98.9,2023-02-26,1,A32TK-12,100.00,100.00,100.00,100.00,100.00,97.75
27,None,A32TK,A32TK.1,Q6,2023-02-26,1.0,A32TK-25,97.50,98.75,97.75,98.87,100.0,100.0,2023-02-26,1,A32TK-25,100.00,100.00,100.00,100.00,100.00,96.62
26,None,A32TK,A32TK.1,Q6,2023-02-26,1.0,A32TK-24,90.00,96.25,93.25,97.75,96.7,98.9,2023-02-26,1,A32TK-24,100.00,98.87,98.87,98.87,96.62,94.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17,1,None,100.00,95.50,94.38,97.75,97.75,96.62
11,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17,1,None,96.62,96.62,95.50,98.87,98.87,96.62
12,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17,1,None,97.75,97.75,95.50,100.00,98.87,92.13
13,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17,1,None,100.00,98.87,97.75,100.00,97.75,96.62


In [2]:
DAYSBACK = 300

DD2_QUERY = f"""
WITH lot_name_cte AS(
    SELECT DISTINCT wafer_id, E.CELL_NAME AS Lot_Name
    FROM DMIW_SYSTEMS.WAFER A,
        DMIW_SYSTEMS.PRODUCT B,
        DMIW_SYSTEMS.EWR_EVENT_FACT C,
        DMIW_SYSTEMS.EWR_EVENT D,
        DMIW_SYSTEMS.EWR_CELL E
    WHERE A.WAFERKEY=C.WAFERKEY
        AND B.PRODUCTKEY=C.PRODUCTKEY
        AND  C.EWR_CELLKEY=E.EWR_CELLKEY
        AND OWNER_USER_ID='hai.zhu@ibm.com'
        AND EWR_NAME IN ('7HPP_Name_P10', '7HPP_Name_zA')
        AND (CELL_ID LIKE '7HPP_Name_P10%' OR CELL_ID LIKE '7HPP_Name_zA%')
        AND E.CELL_NAME NOT IN ('Lot0-1','Lot0-2','uncertain')
),
sail_cte AS(
    SELECT LEFT(wafer_id,5) AS lot_id_base, lot_id, wafer_id, family_code, MIN(tw.Last_test_date) AS Sail_date, 1 AS Sail_n,
      AVG(CASE WHEN parm_Label = 'SAIL23_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p50to1p05,
      AVG(CASE WHEN parm_Label = 'SAIL23_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p65to1p05,
      AVG(CASE WHEN parm_Label = 'SAIL2_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL2_0p50to1p05,
      AVG(CASE WHEN parm_Label = 'SAIL2_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL2_0p65to1p05,
      AVG(CASE WHEN parm_Label = 'SAIL3_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL3_0p50to1p05,
      AVG(CASE WHEN parm_Label = 'SAIL3_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL3_0p65to1p05
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testparmkey = tp.testparmkey
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
    WHERE tw.Last_test_date >= (current date - {DAYSBACK} days)
      AND tw.Tech_id = '7HPP'
      AND tw.Level = 'H2'
      AND tw.Calcdefs in ('SSL11', 'SSL21', 'SSL31')
      AND weighted_Mean IS NOT null AND ABS(weighted_Mean) < 1e25
      AND UCASE(tp.parm_Label) LIKE '%_PERFECTYIELD'
    GROUP BY lot_id, wafer_id, family_code
    ORDER BY Sail_date desc, lot_id, wafer_id
),
pele_cte AS(
    SELECT lot_Id, wafer_Id, family_code, MIN(last_Test_Date) AS Pele_date, 1 as Pele_n,
        AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_L3_HL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_L3_Vmax_PY,
        AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_L3_NL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_L3_Vnom_PY,
        AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_L3_LL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_L3_Vmin_PY,
        AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_1K_HL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_1K_Vmax_PY,
        AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_1K_NL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_1K_Vnom_PY,
        AVG(CASE WHEN ucase(tp.parm_Label) = 'PELE1_H2_1K_LL_PERFECTYIELD' THEN yield ELSE null END) AS PELE1_1K_Vmin_PY
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = ptwf.testParmKey 
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days) AND
        Tech_id = '7HPP' and Level = 'H2' AND tw.Calcdefs IN ('PEL11', 'PEL21', 'PEL31') AND
        weighted_Mean IS NOT null AND abs(weighted_Mean) < 1e25 AND
        (ucase(tp.parm_Label) LIKE 'PELE%_H2_%PERFECTYIELD')
    GROUP BY lot_Id, wafer_Id, family_code
)
SELECT ln.lot_name, s.lot_id_base, s.lot_id, s.family_code, Sail_date, Sail_n, s.wafer_id, 
    SAILALL_0p50to1p05, SAILALL_0p65to1p05,
    SAIL2_0p50to1p05, SAIL2_0p65to1p05, SAIL3_0p50to1p05, SAIL3_0p65to1p05,
    Pele_date, Pele_n, s.wafer_id AS wafer_id_2,
    PELE1_L3_Vmax_PY, PELE1_L3_Vnom_PY, PELE1_L3_Vmin_PY,
    PELE1_1K_Vmax_PY, PELE1_1K_Vnom_PY, PELE1_1K_Vmin_PY
FROM sail_cte s
FULL OUTER JOIN pele_cte p ON s.wafer_id = p.wafer_id
LEFT JOIN lot_name_cte ln ON s.wafer_id = ln.wafer_id
ORDER BY Sail_date desc, s.lot_id, s.wafer_id
"""

In [3]:
def dd2_get_wafer_data():
    w_df = ibmdata.isdw.query(DD2_QUERY)

    w_df = w_df.rename(columns={'wafer_id_2': 'wafer_id'})
    w_df['sailall_0p50to1p05'] = w_df['sailall_0p50to1p05'].astype(float).round(2)
    w_df['sailall_0p65to1p05'] = w_df['sailall_0p65to1p05'].astype(float).round(2)
    w_df['sail2_0p50to1p05'] = w_df['sail2_0p50to1p05'].astype(float).round(2)
    w_df['sail2_0p65to1p05'] = w_df['sail2_0p65to1p05'].astype(float).round(2)
    w_df['sail3_0p50to1p05'] = w_df['sail3_0p50to1p05'].astype(float).round(2)
    w_df['sail3_0p65to1p05'] = w_df['sail3_0p65to1p05'].astype(float).round(2)

    w_df['pele1_l3_vmax_py'] = w_df['pele1_l3_vmax_py'].astype(float).round(2)
    w_df['pele1_l3_vnom_py'] = w_df['pele1_l3_vnom_py'].astype(float).round(2)
    w_df['pele1_l3_vmin_py'] = w_df['pele1_l3_vmin_py'].astype(float).round(2)
    w_df['pele1_1k_vmax_py'] = w_df['pele1_1k_vmax_py'].astype(float).round(2)
    w_df['pele1_1k_vnom_py'] = w_df['pele1_1k_vnom_py'].astype(float).round(2)
    w_df['pele1_1k_vmin_py'] = w_df['pele1_1k_vmin_py'].astype(float).round(2)

    return w_df.sort_values('sail_date',ascending=False)

In [4]:
dd2_get_wafer_data()

,lot_name,lot_id_base,lot_id,family_code,sail_date,sail_n,wafer_id,sailall_0p50to1p05,sailall_0p65to1p05,sail2_0p50to1p05,sail2_0p65to1p05,sail3_0p50to1p05,sail3_0p65to1p05,pele_date,pele_n,wafer_id,pele1_l3_vmax_py,pele1_l3_vnom_py,pele1_l3_vmin_py,pele1_1k_vmax_py,pele1_1k_vnom_py,pele1_1k_vmin_py
21,None,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-02,88.75,92.50,94.31,96.59,93.4,94.5,2021-03-01,1.0,AZA7P-02,96.62,96.62,96.62,98.87,98.87,97.75
23,None,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-06,86.25,92.50,96.59,98.86,90.1,93.4,2021-03-01,1.0,AZA7P-06,96.62,95.50,95.50,97.75,97.75,97.75
24,None,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-07,87.50,88.75,96.59,96.59,91.2,92.3,2021-03-01,1.0,AZA7P-07,97.75,96.62,96.62,100.00,98.87,97.75
25,None,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-09,87.34,89.87,92.04,94.31,93.4,93.4,2021-03-01,1.0,AZA7P-09,97.75,97.75,97.75,97.75,97.75,97.75
22,None,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-04,84.81,91.13,94.31,96.59,92.3,95.6,2021-03-01,1.0,AZA7P-04,96.62,95.50,95.50,98.87,98.87,97.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-29,1.0,None,95.23,95.23,95.23,96.42,95.23,95.23
17,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-29,1.0,None,97.61,96.42,96.42,94.04,92.85,92.85
18,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-29,1.0,None,98.80,97.61,95.23,97.61,96.42,96.42
19,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-29,1.0,None,94.04,92.85,94.04,96.42,96.42,96.42


In [5]:
def dd2_get_lot_data():
    # Lot Level df
    temp_agg_df = dd2_get_wafer_data()
    temp_agg_df['sail_date'] = temp_agg_df['sail_date'].fillna(datetime.date(9999, 12, 30))
    temp_agg_df['pele_date'] = temp_agg_df['pele_date'].fillna(datetime.date(9999, 12, 30))
    temp_agg_df['lot_name'] = temp_agg_df['lot_name'].fillna("")

    lot_df = temp_agg_df.groupby(['lot_name', 'lot_id_base', 'lot_id', 'family_code']).agg({
        'sail_date': 'min', 'sail_n': 'sum',
        'sailall_0p50to1p05': 'mean', 'sailall_0p65to1p05': 'mean',
        'sail2_0p50to1p05': 'mean', 'sail2_0p65to1p05': 'mean',
        'sail3_0p50to1p05': 'mean', 'sail3_0p65to1p05': 'mean',
        'pele_date': 'min', 'pele_n': 'sum',
        'pele1_l3_vmax_py': 'mean', 'pele1_l3_vnom_py': 'mean',
        'pele1_l3_vmin_py': 'mean', 'pele1_1k_vmax_py': 'mean',
        'pele1_1k_vnom_py': 'mean', 'pele1_1k_vmin_py': 'mean'
    }).round(2).sort_values('sail_date', ascending=False).reset_index()

    # Add lot_id columns in the middle of the table
    lot_df.insert(12, 'lot_id_2', lot_df.lot_id)
    lot_ids = lot_df.lot_id
    lot_df = lot_df.drop(columns=['lot_id'])
    lot_df.insert(5, 'lot_id', lot_ids)
    lot_df = lot_df.rename(columns={'lot_id_2': 'lot_id', 'lot_id_3': 'lot_id'})
    lot_df['sail_date'] = lot_df['sail_date'].replace(datetime.date(9999, 12, 30),np.nan)
    lot_df['pele_date'] = lot_df['pele_date'].replace(datetime.date(9999, 12, 30),np.nan)
    
    return lot_df

In [6]:
dd2_get_lot_data()

,lot_name,lot_id_base,family_code,sail_date,sail_n,lot_id,sailall_0p50to1p05,sailall_0p65to1p05,sail2_0p50to1p05,sail2_0p65to1p05,sail3_0p50to1p05,sail3_0p65to1p05,lot_id,pele_date,pele_n,pele1_l3_vmax_py,pele1_l3_vnom_py,pele1_l3_vmin_py,pele1_1k_vmax_py,pele1_1k_vnom_py,pele1_1k_vmin_py
0,,AZA7P,Q6,2021-02-28,5.0,AZA7PB.1,86.93,90.95,94.77,96.59,92.08,93.84,AZA7PB.1,2021-03-01,5.0,97.07,96.40,96.40,98.65,98.42,97.75
1,,AZA62,Q6,2021-02-23,5.0,AZA62A.1,78.44,93.23,91.13,98.40,87.47,94.50,AZA62A.1,2021-02-23,5.0,97.52,96.85,96.62,99.10,98.65,98.42
2,,AZA29,Q6,2021-02-20,2.0,AZA29.1,65.62,70.62,70.45,76.70,90.10,91.20,AZA29.1,2021-02-20,2.0,86.51,82.02,80.33,78.09,67.41,62.35
3,CS27,AZ8BY,Q5,2021-02-17,15.0,AZ8BY.1,90.08,91.22,96.42,97.13,93.10,93.48,AZ8BY.1,2021-02-17,15.0,98.80,98.49,98.17,98.48,98.01,97.69
4,ER29,AZ92Y,Q5,2021-02-08,8.0,AZ92Y.1,73.35,95.88,85.86,97.76,85.77,97.56,AZ92Y.1,2021-02-08,8.0,96.87,95.97,94.49,97.91,97.31,96.57
5,"ER ""DP2""(DP#5)",AZ81Y,Q5,2021-01-22,20.0,AZ81Y.1,94.07,95.65,97.49,98.33,96.44,97.07,AZ81Y.1,2021-01-22,20.0,97.19,94.87,93.45,98.03,95.88,93.92
6,ER28A (wasCS28),AZ8CK,Q5,2021-01-11,14.0,AZ8CK.1,92.94,94.26,97.02,97.61,95.89,96.47,AZ8CK.1,2021-01-11,14.0,99.15,98.72,98.72,99.32,99.23,99.06
7,ER28B (wasCS28),AZ8CK,Q5,2021-01-10,10.0,AZ8CKA.1,90.26,91.97,96.18,97.13,92.41,92.98,AZ8CKA.1,2021-01-10,10.0,99.16,99.04,98.92,99.04,98.92,97.37
8,CS24,AZ884,Q5,2020-12-28,24.0,AZ884.1,92.21,93.20,96.32,96.92,94.54,95.40,AZ884.1,2020-12-29,24.0,99.15,98.50,97.31,99.30,99.00,98.20
9,ER Unknown,AZ7UT,Q5,2020-12-18,17.0,AZ7UTB.1,94.58,95.74,96.91,97.47,96.55,97.16,AZ7UTB.1,2020-12-19,17.0,99.65,99.58,99.37,99.65,99.08,98.31


In [7]:
def dd2_get_macro_data(selected_item: str, n: int):
    w_df = dd2_get_wafer_data()
    w_df['lot_name'] = w_df['lot_name'].fillna("")

    if selected_item == 'sail':
        parmLabels = [
            'lot_name','lot_id_base','sail_date',
            'sailall_0p50to1p05','sailall_0p65to1p05',
            'sail2_0p50to1p05','sail2_0p65to1p05',
            'sail3_0p50to1p05','sail3_0p65to1p05'
        ]
        sortDate = 'sail_date'

    elif selected_item == 'pele':
        parmLabels = [
            'lot_name','lot_id_base','pele_date',
            'pele1_l3_vmax_py','pele1_l3_vnom_py','pele1_l3_vmin_py',
            'pele1_1k_vmax_py','pele1_1k_vnom_py','pele1_1k_vmin_py'
        ]
        sortDate = 'pele_date'

    lot_ids = w_df[parmLabels].dropna().sort_values(sortDate,ascending=False).lot_id_base.unique()[0:n]
    w_df_temp = w_df.loc[:,~w_df.columns.duplicated()].loc[w_df['lot_id_base'].isin(lot_ids)].sort_values(sortDate,ascending=True)
    w_df_temp['xlabel'] = np.where(w_df_temp.lot_name == "", w_df_temp.lot_id_base, w_df_temp.lot_id_base +" ("+ w_df_temp.lot_name+")")

    return w_df_temp

In [8]:
dd2_get_macro_data('pele',10)

,lot_name,lot_id_base,lot_id,family_code,sail_date,sail_n,wafer_id,sailall_0p50to1p05,sailall_0p65to1p05,sail2_0p50to1p05,sail2_0p65to1p05,sail3_0p50to1p05,sail3_0p65to1p05,pele_date,pele_n,pele1_l3_vmax_py,pele1_l3_vnom_py,pele1_l3_vmin_py,pele1_1k_vmax_py,pele1_1k_vnom_py,pele1_1k_vmin_py,xlabel
153,CS22,AZ797,AZ797.1,Q5,2020-12-09,1.0,AZ797-01,90.78,90.78,98.80,98.80,91.95,91.95,2020-12-10,1.0,100.00,100.00,100.00,98.80,98.80,98.80,AZ797 (CS22)
143,CS22,AZ797,AZ797.1,Q5,2020-12-10,1.0,AZ797-16,98.68,98.68,100.00,100.00,98.85,98.85,2020-12-10,1.0,98.80,98.80,97.61,100.00,100.00,100.00,AZ797 (CS22)
142,CS22,AZ797,AZ797.1,Q5,2020-12-10,1.0,AZ797-15,90.78,92.10,94.04,95.23,94.25,94.25,2020-12-10,1.0,100.00,100.00,100.00,97.61,97.61,96.42,AZ797 (CS22)
141,CS22,AZ797,AZ797.1,Q5,2020-12-10,1.0,AZ797-14,98.68,98.68,100.00,100.00,98.85,98.85,2020-12-10,1.0,98.80,98.80,97.61,98.80,97.61,97.61,AZ797 (CS22)
159,CS22,AZ797,AZ797.1,Q5,2020-12-09,1.0,AZ797-07,92.10,92.10,96.42,96.42,94.25,94.25,2020-12-10,1.0,98.80,98.80,98.80,97.61,97.61,96.42,AZ797 (CS22)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-04,84.81,91.13,94.31,96.59,92.30,95.60,2021-03-01,1.0,96.62,95.50,95.50,98.87,98.87,97.75,AZA7P
25,,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-09,87.34,89.87,92.04,94.31,93.40,93.40,2021-03-01,1.0,97.75,97.75,97.75,97.75,97.75,97.75,AZA7P
24,,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-07,87.50,88.75,96.59,96.59,91.20,92.30,2021-03-01,1.0,97.75,96.62,96.62,100.00,98.87,97.75,AZA7P
23,,AZA7P,AZA7PB.1,Q6,2021-02-28,1.0,AZA7P-06,86.25,92.50,96.59,98.86,90.10,93.40,2021-03-01,1.0,96.62,95.50,95.50,97.75,97.75,97.75,AZA7P
